# UKWA Stats Module

Copyright 2023 Michael George (AKA Logiqx).

This file is part of [wsw-results](https://github.com/Logiqx/wsw-results) and is distributed under the terms of the GNU General Public License.

wsw-results is free software: you can redistribute it and/or modify it under the terms of the GNU General Public License as published by the Free Software Foundation, either version 3 of the License, or (at your option) any later version.

wsw-results is distributed in the hope that it will be useful, but WITHOUT ANY WARRANTY; without even the implied warranty of MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE. See the GNU General Public License for more details.

You should have received a copy of the GNU General Public License along with sse-results. If not, see <https://www.gnu.org/licenses/>.

### Summary

Adhoc script to determine the correctness of UKWA flags in the current year

## Initialisation

Basic approach to determine the project directory

In [1]:
import os
import sys
import glob

import csv
import math

import numpy as np

from datetime import datetime

In [2]:
PROJDIR = os.path.realpath(os.path.join(sys.path[0], '..'))

In [3]:
RUN_2_KTS = 'Run 2 (kts)'
AVG_KTS = 'Avg (kts)'
STATUS = 'Status'

## Process Entrants

Process all names

In [4]:
def load_courses():
    '''Load courses'''
       
    csvFile = os.path.join(PROJDIR, 'resources', 'ukwa', 'courses.csv')
    
    courses = {}
    with open(csvFile, 'r', encoding='utf-8') as f:
        csvReader = csv.DictReader(f)
        for row in csvReader:
            courses[row['session']] = row['course']

    return courses

In [5]:
def isValidOld(results):
    '''Evaluate results using old rules'''
    
    allRuns = []
    for result in results:
        try:
            allRuns.append(float(result[RUN_2_KTS]))
        except ValueError:
            allRuns.append(0)
            
    allRuns = sorted(allRuns, reverse=True)
            
    percent25 = math.ceil(len(allRuns) / 4)
    percent50 = math.ceil(len(allRuns) / 2)
    
    valid = False

    if allRuns[percent25 - 1] >= 28:
        valid = True
        
    return valid
    

def isValidNew(results, course):
    '''Evaluate results using new rules'''

    silverRuns = []
    for result in results:
        if 'gold' in result[STATUS].lower() or 'pro' in result[STATUS].lower():
            try:
                silverRuns.append(float(result[RUN_2_KTS]))
            except ValueError:
                silverRuns.append(0)

    valid = False

    if len(silverRuns) > 0:
        silverRuns = sorted(silverRuns, reverse=True)

        percent25 = math.ceil(len(silverRuns) / 4)
        percent50 = math.ceil(len(silverRuns) / 2)

        if course == 'H':
            threshold = 27
        else:
            threshold = 28

        if silverRuns[percent25 - 1] >= threshold:
            valid = True
        
    return valid

    
def processSession(year, csvFile, courses):
    '''processSession processes an individual session for an event'''

    results = []
    with open(csvFile, 'r', encoding='utf-8') as f:
        csvReader = csv.DictReader(f)
        for result in csvReader:
            results.append(result)

    session = os.path.basename(os.path.dirname(csvFile))
    course = courses[session]
    
    validOld = isValidOld(results)
    validNew = isValidNew(results, course)

    baseUrl = 'https://logiqx.github.io/wsw-results/results'
    link = '[{} / {} / {}]({}/{}/{}/ukwa.html)'.format(session, os.path.splitext(os.path.basename(csvFile))[0], course,
                                                     baseUrl, session[:4], session)

    if validOld == True and validNew == False:
        print('-', link, '- becomes invalid')
    elif validOld == False and validNew == True:
        print('-', link, '- becomes valid')

In [6]:
def processEvent(year, courses):
    '''processYear processes all of the sessions for an event'''

    csvFiles = sorted(glob.glob(os.path.join(PROJDIR, 'events', str(year), 'sailwave', '*', '*Men*.csv')))
    
    for csvFile in csvFiles:
        if "Wing" not in csvFile:
            processSession(year, csvFile, courses)

In [7]:
def main():
    courses = load_courses()
    
    for year in range(2010, 2030):
        processEvent(year, courses)

    print('\nAll done')

if __name__ == '__main__':
    main()

- [20121009 / Leg 1 - Men / H](https://logiqx.github.io/wsw-results/results/2012/20121009/ukwa.html) - becomes valid
- [20121009 / Leg 2 - Men / H](https://logiqx.github.io/wsw-results/results/2012/20121009/ukwa.html) - becomes valid
- [20121010 / Leg 2 - Men / H](https://logiqx.github.io/wsw-results/results/2012/20121010/ukwa.html) - becomes valid
- [20141018 / Leg 1 - Men / S](https://logiqx.github.io/wsw-results/results/2014/20141018/ukwa.html) - becomes valid
- [20141018 / Leg 2 - Men / S](https://logiqx.github.io/wsw-results/results/2014/20141018/ukwa.html) - becomes valid
- [20151004 / Leg 2 - Men / H](https://logiqx.github.io/wsw-results/results/2015/20151004/ukwa.html) - becomes valid
- [20151005 / Leg 1 - Men / H](https://logiqx.github.io/wsw-results/results/2015/20151005/ukwa.html) - becomes valid
- [20161012 / Leg 1 - Men / H](https://logiqx.github.io/wsw-results/results/2016/20161012/ukwa.html) - becomes valid
- [20161012 / Leg 2 - Men / H](https://logiqx.github.io/wsw-resu

## All Done!